# Web Scrapping

## Importing Libraries

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
from time import sleep

## Scraping ATP Top 100 Player Profiles

In [2]:
def parse_soup(sp):
    label_temp = []
    value = []

    # Players name and appending to value
    first_name = sp.find('div', class_='first-name').text
    last_name = sp.find('div', class_='last-name').text
    player_name = first_name + ' ' + last_name
    value.append(player_name)

    # This is pulling the Rank label and if it is for singles or doubles
    data_label_rank = sp.find('div', class_='data-label-text').text.strip()
    data_label_rank = re.sub('\r\n', ' ', data_label_rank)
    label_temp.append((re.sub(' +', ' ', data_label_rank)))

    # This is pull the value for the rank label
    data_number = sp.find('div', class_='data-number').text.strip()
    value.append(data_number)
    # rank = data_number  # Variable for class

    # This is pulling the labels Age, Turned Pro, Weight, and Height.
    for table_big_label in sp.find_all('div', class_='table-big-label'):
        label_temp.append(table_big_label.text)

    # This is pulling the values for the labels above

    for table_big_value in sp.find_all('div', class_='table-big-value'):
        x = table_big_value.text.strip()
        bad = "\n\r\n"
        for r in bad:
            x = x.replace(r, "")
        value.append(re.sub(' +', ' ', x))

    # This is pulling labels for Birthplace, Plays, Coach
    for table_label in sp.find_all('div', class_='table-label'):
        label_temp.append(table_label.text.strip())

    # This is pulling labels for Birthplace, Plays, Coach
    for table_value in sp.find_all('div', class_='table-value'):
        value.append(table_value.text.strip())

    return value

def player(value):

    # This loop is taking the string of variables and assigning them to their respected values
    label = ['fullname', 'rank', 'age', 'turned_pro', 'weight', 'height', 'birthplace', 'plays', 'coach']
    n = 0
    for name in label:
        globals()[name] = value[n]
        n += 1
    return fullname, rank, age, turned_pro, weight, height, birthplace, plays, coach

In [3]:
value_list = []  # Empty list to insert each player into in
html_text_rank = requests.get('https://www.atptour.com/en/rankings/singles').text
soup_rank = BeautifulSoup(html_text_rank, 'lxml')

url_link = []
html_1 = 'https://www.atptour.com'  # Needed this to add the link a pulled in from singles rank page

# This for loop is grabbing the links to the players overview if the link doesn't contain overview it passes
for a in soup_rank.find_all('a', href=True):
    link = a['href']
    if 'overview' in link:
        url = html_1 + link
        url_link.append(url)
    else:
        pass
print('all links grabbed')
# this for loop runs the functions above to grab the info from overview for each player and appends to value_list
for i in url_link:
    html_text = requests.get(i).text
    soup = BeautifulSoup(html_text, 'lxml')
    player(parse_soup(soup))
    value_list.append((fullname, int(rank), age, turned_pro, weight, height, birthplace, plays, coach))
    print(fullname, 'added to list')
# This is creating the dataframe
df = pd.DataFrame(value_list, columns=['fullname', 'rank', 'age', 'turned_pro', 'weight', 'height', 'birthplace', 'plays', 'coach'])
df.head()    

all links grabbed
Carlos Alcaraz added to list
Rafael Nadal added to list
Casper Ruud added to list
Stefanos Tsitsipas added to list
Novak Djokovic added to list
Andrey Rublev added to list
Felix Auger-Aliassime added to list
Daniil Medvedev added to list
Taylor Fritz added to list
Holger Rune added to list
Hubert Hurkacz added to list
Cameron Norrie added to list
Alexander Zverev added to list
Matteo Berrettini added to list
Pablo Carreno Busta added to list
Jannik Sinner added to list
Frances Tiafoe added to list
Marin Cilic added to list
Lorenzo Musetti added to list
Karen Khachanov added to list
Nick Kyrgios added to list
Denis Shapovalov added to list
Borna Coric added to list
Alex de Minaur added to list
Roberto Bautista Agut added to list
Diego Schwartzman added to list
Miomir Kecmanovic added to list
Grigor Dimitrov added to list
Francisco Cerundolo added to list
Daniel Evans added to list
Sebastian Korda added to list
Alejandro Davidovich Fokina added to list
Yoshihito Nishiok

,fullname,rank,age,turned_pro,weight,height,birthplace,plays,coach
0,Carlos Alcaraz,1,19 (2003.05.05),2018,163lbs(74kg),"6'0""(183cm)","El Palmar, Murcia, Spain","Right-Handed, Two-Handed Backhand",Juan Carlos Ferrero
1,Rafael Nadal,2,36 (1986.06.03),2001,187lbs(85kg),"6'1""(185cm)","Manacor, Mallorca, Spain","Left-Handed, Two-Handed Backhand","Carlos Moya, Marc Lopez, Gustavo Marcaccio"
2,Casper Ruud,3,24 (1998.12.22),2015,170lbs(77kg),"6'0""(183cm)","Oslo, Norway","Right-Handed, Two-Handed Backhand",Christian Ruud
3,Stefanos Tsitsipas,4,24 (1998.08.12),2016,198lbs(90kg),"6'4""(193cm)","Athens, Greece","Right-Handed, One-Handed Backhand","Apostolos Tsitsipas, Mark Philippoussis"
4,Novak Djokovic,5,35 (1987.05.22),2003,170lbs(77kg),"6'2""(188cm)","Belgrade, Serbia","Right-Handed, Two-Handed Backhand",Goran Ivanisevic


## Scraping Livescore.in for point by point data

In [9]:
%%time
# This function works on livescore.in and grabs the links to all of the tournaments in the archive tab.
# No need to run. Works but is not clean, the next cell is the better on to run.
main = []
tennis = "/tennis/atp-singles/"

url = "https://www.livescore.in/tennis/atp-singles/"
url_base = "https://www.livescore.in"
# get html content
r = requests.get(url)
html_content_main = r.text
# print(html_content)
# parse the html using BeautifulSoup
soup = BeautifulSoup(html_content_main, "html.parser")
# we are looking for all atp-singles matches first
for i in soup.find_all("a", href=True):
    # this holder list will eventually be appended to the main list
    # going to be holding all of the main links to match data
    holder = []
    extension = i['href']
    if tennis in extension:
        new_url = url_base + extension
        # sending our first main link to the holder list
        holder.append(new_url)
        r_level_1 = requests.get(new_url)
        html_content_level_1 = r_level_1.text
        soup = BeautifulSoup(html_content_level_1, "html.parser")
        # going into each of the tournaments and looking for the archive tab
        for i in soup.find_all("a", href=True):
            extension_2 = i['href']
            # looking to see which links contain 'archive'
            if 'archive' in extension_2:
                arch_url = url_base + extension_2
                # grabbing those links html and looking for past tournament years
                r_level_2 = requests.get(arch_url)
                html_content_level_2 = r_level_2.text
                soup = BeautifulSoup(html_content_level_2, "html.parser")
                # we are we are going through the links in the archive tab
                # and only collecting past years in the 2000s which all have a -20
                for i in soup.find_all("a", href=True):
                    match_year_url = i['href']
                    if '-20' in match_year_url:
                        holder.append(match_year_url)
                    else:
                        pass
            else:
                pass
    else:
        pass
    main.append(holder)


CPU times: user 18.8 s, sys: 374 ms, total: 19.2 s
Wall time: 3min 8s


In [22]:
# This is an optimized and better output than the cell above... keeping both for reference for now

main = []
tennis = "/tennis/atp-singles/"

url = "https://www.livescore.in/tennis/atp-singles/"
url_base = "https://www.livescore.in"
# get html content
r = requests.get(url)
html_content_main = r.text
# parse the html using BeautifulSoup
soup = BeautifulSoup(html_content_main, "html.parser")
# we are looking for all atp-singles matches first
holder = [url_base + i['href'] for i in soup.find_all("a", href=True) if tennis in i['href']]
# level 1
for new_url in holder:
    main.append(new_url)
    r_level_1 = requests.get(new_url)
    html_content_level_1 = r_level_1.text
    soup = BeautifulSoup(html_content_level_1, "html.parser")
    # level 2
    if len([url_base + i['href'] for i in soup.find_all("a", href=True) if 'archive' in i['href']])>0:
        arch_url = [url_base + i['href'] for i in soup.find_all("a", href=True) if 'archive' in i['href']][0]
        r_level_2 = requests.get(arch_url)
        html_content_level_2 = r_level_2.text
        soup = BeautifulSoup(html_content_level_2, "html.parser")
        # level 3
        match_year_url = [i['href'] for i in soup.find_all("a", href=True) if '-20' in i['href']]
        for i in match_year_url:
            final_url = url_base + i
            main.append(final_url)
    print(f'{new_url} has been stored')


https://www.livescore.in/tennis/atp-singles/ has been stored
https://www.livescore.in/tennis/atp-singles/australian-open/ has been stored
https://www.livescore.in/tennis/atp-singles/ has been stored
https://www.livescore.in/tennis/atp-singles/adelaide/ has been stored
https://www.livescore.in/tennis/atp-singles/amersfoort/ has been stored
https://www.livescore.in/tennis/atp-singles/antalya/ has been stored
https://www.livescore.in/tennis/atp-singles/antwerp/ has been stored
https://www.livescore.in/tennis/atp-singles/atp-cup/ has been stored
https://www.livescore.in/tennis/atp-singles/auckland/ has been stored
https://www.livescore.in/tennis/atp-singles/australian-open/ has been stored
https://www.livescore.in/tennis/atp-singles/bangkok/ has been stored
https://www.livescore.in/tennis/atp-singles/barcelona/ has been stored
https://www.livescore.in/tennis/atp-singles/basel/ has been stored
https://www.livescore.in/tennis/atp-singles/bastad/ has been stored
https://www.livescore.in/tenni

In [24]:
with open("output.txt", "w") as f:
    f.writelines(str(i) + '\n' for i in main)

In [5]:
url = "https://www.livescore.in/tennis/atp-singles/halle/"
_temp = []
# get html content
r = requests.get(url)
html_content = r.text
soup = BeautifulSoup(html_content, "html.parser")

# find class="lmc__template"
for i in soup.find_all("a", href=True):
    link = i['href']
    if 'archive' in link:
        _temp.append(link)
    else:
        pass
print(_temp)


['/tennis/atp-singles/halle/archive/']


In [6]:
url = "https://www.livescore.in/tennis/atp-singles/halle/archive/"
_temp = []
# get html content
r = requests.get(url)
html_content = r.text
soup = BeautifulSoup(html_content, "html.parser")

# find class="lmc__template"
for i in soup.find_all("a", href=True):
    link = i['href']
    if '/halle-' in link:
        _temp.append(link)
    else:
        pass
print(_temp)

['/tennis/atp-singles/halle-2021/', '/tennis/atp-singles/halle-2019/', '/tennis/atp-singles/halle-2018/', '/tennis/atp-singles/halle-2017/', '/tennis/atp-singles/halle-2016/', '/tennis/atp-singles/halle-2015/', '/tennis/atp-singles/halle-2014/', '/tennis/atp-singles/halle-2013/', '/tennis/atp-singles/halle-2012/', '/tennis/atp-singles/halle-2011/', '/tennis/atp-singles/halle-2010/', '/tennis/atp-singles/halle-2009/', '/tennis/atp-singles/halle-2008/', '/tennis/atp-singles/halle-2007/', '/tennis/atp-singles/halle-2006/', '/tennis/atp-singles/halle-2005/', '/tennis/atp-singles/halle-2004/', '/tennis/atp-singles/halle-2003/', '/tennis/atp-singles/halle-2002/', '/tennis/atp-singles/halle-2001/', '/tennis/atp-singles/halle-2000/', '/tennis/atp-singles/halle-1999/', '/tennis/atp-singles/halle-1998/', '/tennis/atp-singles/halle-1997/', '/tennis/atp-singles/halle-1996/', '/tennis/atp-singles/halle-1995/', '/tennis/atp-singles/halle-1994/', '/tennis/atp-singles/halle-1993/']
